# Lab Manual for Project-1-PyPair

This lab manual provides an overview of the project, explains the functionality of each section, and guides you on how to run the game.

## Overview
Project-1-PyPair is a Python-based text adventure game where players fight demons, collect items, and progress through a storyline. The game demonstrates the use of Python classes, JSON handling, file operations, and modular programming.

## Key Components

### 1. **Player Classes**
- Located in `Player.py`.
- Defines the player character classes: `Mage`, `Warrior`, `Shadow`, and `Archer`.
- Each class has unique attributes and level-up mechanics.

#### Example Code:

In [ ]:
# filepath: ~~\project-1-pypair\Project_Main_Function\Player.py
class Mage(Player):
    def __init__(self, name):
        super().__init__(name, 80, 150, 60, 6, 10, 18)
        self.magic_power = 25  # Unique attribute
    def level_up(self):
        super().level_up()
        self.hp += 5
        self.mana += 15
        self.intelligence += 5

### 2. **Inventory System**
- Located in `Inventory.py`.
- Defines the Item as a dictionary, with name, type, rarity.
- Use logging as error handler for most case

#### Function `display_inventory(inventory)`
- Handle displaying the inventory with looping, showing name, rarity, and if a trigger word, `damage`, `defense`, or `effect` is in then print out different information.

#### Function `use_item(player)`
- Select the item in the inventory that log if the inventory is empty.
- Select item to use using enumerate the inventory from 1 or 0 to exit.
- Check if it's a consumable item/ equippable using keyword (`effect`, `damage`), since all items are listed this way.

#### Function `apply_item_effect(player, item)`
- This function calls whenever an item from `use_item` item have an `effect` keyword, applying the effect of healing since we don't have others for now.

#### Function `meets_requirements(player,item)`
- This function checks if a player meets the requirement to equip an item.
- This function is calls whenever there is a `damage` keyword in the `use_item` function.

#### Function `equip_item(player, item)`
- Calls after the `meets_requirements` function, check if the item is armor or weapon.
- If `damage` is in keyword -> weapons, if `defense` is in keyword -> armor

#### Function `drop_item(player, item_name)`
- This allows player to drop an item from their inventory
- Not yet implemented.

In [ ]:
def display_inventory(inventory):
    """Displays the player's inventory."""
    print("\nPlayer Inventory:")
    print("=" * 30)
    if not inventory:
        print("Your inventory is empty.")
    for idx, item in enumerate(inventory, 1):
        print(f"{idx}. {item['name']} ({item['rarity']})")
        print(f"   Type: {item['type']}")
        if 'damage' in item:
            print(f"   Damage: {item['damage']}")
        if 'defense' in item:
            print(f"   Defense: {item['defense']}")
        if 'effect' in item:
            print(f"   Effect: {item['effect']}")
        print(f"   Requirement: {item['requirement']}")
        print(f"   {item['description']}")
        print("-" * 30)

### 3. **Utilities (Ultilities)**
- Don't mind the wrong spelling, we couldn't spell and too lazy to change.
- Logging is use to raise all our errors

#### Function `slow_prints(text, delay=0.05)`
- This is use widely throughout our code, since we want to add *immersion* into our game.
- This prints out text one character at a time with a delay.
- Default is to 0.05 seconds.
- Uses a loop with sys library to flush out characters.
```python
def slow_print(text, delay=0.05):
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    print()
```

#### Function `clear_screen()`
- Using os library to clear the terminal screen for better readibility.
```python
def clear_screen():
    os.system("cls" if os.name == "nt" else "clear")
```

#### Function `wrap_text(text, width=80)`
- This wraps text to specified width for better visibility
```python
def wrap_text(text, width=80):
    """Wraps text to a specified width for better display."""
    return "\n".join(textwrap.wrap(text, width))
```

### 4. **Game Story**
- This script is use to navigate json file and move the story along.

#### Function `load_story()`
- Load in the storyline_path using os.path.join since our directories are not setup nicely.
- use try and except with errors catching since this is a big issue if the story cannot be load.

```python
def load_story():
    storyline_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), "Project_Json", "Storyline.json")
    try:
        with open(storyline_path, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        print("Error: Storyline file not found.")
        return {}
    except json.JSONDecodeError:
        print("Error: Invalid JSON format in storyline.")
        return {}
```


#### Function `progress_story(player,node,storyline)`
- This function uses the json file as stem, grows with nodes that contain `event`.
- Raises the error if node not found (big issue).
```python
if node not in storyline:
    slow_print("Invalid story path.", delay=0.02)
    return
```
- The event is what node the character is on.
```python
event = storyline[node]
```
- We set tier ranges based on node index
```py
    # Tier ranges based on node index
    if node_index >= 10:  # Tier 3 starts at index 10
        tier = 3
    elif node_index >= 7:  # Tier 2 starts at index 7
        tier = 2 
    elif node_index >= 4:  # Tier 1 starts at index 4
        tier = 1
```

#### Function `start_story(player)`
- Loads Storyline: Retrieves the game's storyline using the `load_story()` function.
- Handles Pre-Game Interactions: Provides a menu for the player to interact with before starting the adventure.
- Inventory Management: Allows the player to check their inventory via `display_inventory()`.
- Item Usage: Enables the player to use an item by providing its name, calling `use_item()`.
- Game Start Handling: Checks if a "menu_link" exists in the storyline and begins the adventure using `progress_story()`.
- Exit Handling: Allows the player to exit the game gracefully with a farewell message.
- Invalid Input Handling: Prints an error message if the player enters an unrecognized command.
- Action Mapping: Maps user choices `(start, inventory, use item, exit)` to corresponding functions.
- Loop for Interaction: Continuously prompts the player until they choose to start the game or exit.

```py
def start_story(player):
    """Starts the storyline with the provided player instance."""
    storyline = load_story()

    def handle_inventory():
        display_inventory(player.inventory)

    def handle_exit():
        slow_print("Exiting game...", delay=0.02)
        return True

    def handle_use_item():
        item_name = input("Enter the item name to use: ").strip()
        use_item(player, item_name)

    def handle_start():
        if "menu_link" in storyline:
            progress_story(player, "menu_link", storyline)
        else:
            logging.error("Storyline not found.")
        return True

    def handle_invalid():
        slow_print("Invalid choice. Try again.", delay=0.02)

    actions = {
        "inventory": handle_inventory,
        "exit": handle_exit,
        "use item": handle_use_item,
        "start": handle_start,
    }

    while True:
        slow_print("\nYou are in town. What would you like to do?", delay=0.02)
        slow_print("[start] Begin your adventure", delay=0.02)
        slow_print("[inventory] Check Inventory", delay=0.02)
        slow_print("[use item] Use an Item", delay=0.02)
        slow_print("[exit] Exit the game", delay=0.02)

        choice = input("Choose an action: ").strip().lower()
        action = actions.get(choice, handle_invalid)
        if action() is True:
            break
```

#### Example Code:

In [ ]:
# Wouldn't work here since mose function are linked through other scripts
# def use_item(player):
# def slow_print(text, delay=0.05): etc....
def progress_story(player, node, storyline):
    if node not in storyline:
        slow_print("Invalid story path.", delay=0.02)
        return
    
    event = storyline[node]

    # Display story text
    clear_screen()
    slow_print(wrap_text(event["text"]), delay=0.02)

    
    # Determine demon tier based on node progression
    node_list = list(storyline.keys())
    node_index = node_list.index(node)
    total_nodes = len(node_list)
    tier = 1

    # Tier ranges based on node index
    if node_index >= 10:  # Tier 3 starts at index 10
        tier = 3
    elif node_index >= 7:  # Tier 2 starts at index 7
        tier = 2 
    elif node_index >= 4:  # Tier 1 starts at index 4
        tier = 1

    # Handle event (combat or item)
    if "event" in event:
        if event["event"] == "combat":
            demon = spawn_demon(f"tier_{tier}")
            if demon:
                combat(player, demon, node)
            else:
                slow_print("No demons available to fight!", delay=0.02)
        elif event["event"] == "item":
            item = event["item"]
            slow_print(f"You obtained {item}!", delay=0.02)
            player.inventory.append(item)

    while True:
        slow_print("\nWhat do you do?")
        for key, value in event.get("choices", {}).items():
            slow_print(f"[{key}] {value}", delay=0.02)

        # Always allow inventory check and item usage
        slow_print("[inventory] Check Inventory", delay=0.02)
        slow_print("[use item] Use an Item", delay=0.02)
        
        choice = input("Choose an action: ").strip().lower()

        if choice == "inventory":
            display_inventory(player.inventory)
            continue  # Loop back to show options again
        elif choice == "use item":
            use_item(player)
            continue  # Allow using multiple items before choosing an action
        elif choice in event.get("choices", {}):
            next_node = event["choices"][choice]
            progress_story(player, next_node, storyline)  # Move forward
            return  # Stop asking for input

        slow_print("Invalid choice. Please try again.", delay=0.02)

### 5. **Combat System**
- Using os library to construct the relative path to the Json file.
- Loads `Items.json` and `Demons.json`

#### Function `spawn_demon(tier)`
- This function spawn with random library in random.choice from the tier that are from the demons_data[tier]
 
#### Function `get_loot_drop(location, rarity = "common")`
- This function handles the item drop pool with each location in game.
- Make sure that the player only get intermediate loot from certain location in game.

#### Function `combat(player, demon, location)`
- This is a huge function that handles the combat between player and a demon.
- Calculations for damage from both player, demon.
- Implement some critical hit chance for combat.
- The `use_item` function from `Inventory` script is called here if the player wanted to use item while battling.
- Take turn combatting each other and check the combat outcome.
```py
  if player.hp > 0:
        print(f"{player.name} defeated {demon['name']}! Gained {demon['exp']} XP!")
        player.gain_experience(demon["exp"])
```
- Give the player experience if player is still alive.
- As the same time gives them random item base on the location using `get_loot_drop` function.
```py
# Determine if an item drops and the rarity of item
        if random.random() < 0.5:  # 50% chance to drop an item
            rarity_levels = ["common", "uncommon", "rare", "epic", "legendary"]
            loot_rarity = random.choices(rarity_levels, weights=[50, 30, 15, 4, 1], k=1)[0]
            loot = get_loot_drop(location, rarity=loot_rarity)

            if loot:
                player.inventory.append(loot)
                print(f"{player.name} found a {loot['name']} ({loot['rarity']})!")
```

#### Function `get_primary_stat(player)`
- This function is use to specify which primary stat to use for the damage calculation.

#### Function `calculate_damage(player, demon)`
- This function calculate the damage and called to the main `combat` function to return the damage.
- Using the `get_primary_stat` function to determine the player damage, and the weapon_damage from `player.equipped weapon` function from `Player` script.
- We use a poor way to calculate damage from each class:
```py
if isinstance(player, Mage):
        return base_damage + weapon_damage + (player.magic_power // 4)
    elif isinstance(player, Warrior):
        return base_damage + weapon_damage + (player.strength // 3)
    elif isinstance(player, Shadow):
        return base_damage + weapon_damage + (player.dexterity // 3)
    elif isinstance(player, Archer):
        return base_damage + weapon_damage + (player.crit_chance // 2)
    return base_damage + weapon_damage
```
- This can be revamp for better consistency, but each class have different special stats that influnence the damage.

### 6. ***Game Start Script***
- This script is use to bind all the other script to each other and maintain our menu and executing options from menus.
- Main function is this one, with all of the other script implement into functions that works inside this main script.
- Running menu, displaying it, handle the choice, and running the whole game.

#### Function `menu_generator(options)`
- This is use to provide a formatted menu option lists.

#### Function `display_menu(options)`
- This uses some other function from `Ultilities` script for cleanliness.

#### Funtion `handle_choice(options)`
- This function handles user input for menu selection, using a while loop.

#### Function `choose_class()`
- This use the `Player` script for each class, handles the class choice.

#### Function `start_storyline(player)`
- This load the storyline.json using the `*Game_Story* script`.

### Function `execute_option(choice)`
- This function executes the selected option.
- Utilize the `choose_class()` and the `start_storyline` function for the class and player to be pass into.

### *Special* Logging
- Logging is configured in multiple files to track errors and game events.
- Logs are displayed in the terminal for debugging purposes.

## How to Run the Game

1. Ensure all required files are in the project directory.
2. Open a terminal and navigate to the project folder.
3. Run the following command:
   ```bash
   python Project_Main_Function/Game_Start.py
   ```
4. Follow the on-screen instructions to play the game.
---
> VSCode Approach Use this if you have VSCode already installed.
1. Ensure all required files are in the project directory
    - Required directory must be: *\project-1-pypair\Project_Json\(ALL THE JSON FILE) and *\project-1-pypair\Project_Main_Function\(ALL THE SCRIPTS)
    - View the sample directories at [PyPair Github](https://github.com/ETE4990-S25/project-1-pypair)
2. Open the `Game_Start.py` script
3. Run the script using your VSCode.

## Lists of Files:

### JSON Files
- **`Demons.json`**: Contains demon data categorized by tiers.
- **`Items.json`**: Contains item data categorized by name, type, and rarity.
- **`Storyline.json`**: Defines the game's narrative structure, node for the game events, and options that will lead the narrative.

These files are loaded dynamically during the game to provide flexibility and scalability.

### Scripts
***REFER TO KEY COMPONENTS FOR BREAK DOWN OF CODE FUNCTIONS***
- **`Combat_System.py`**
- **`Game_Start.py`**
- **`Game_Story.py`**
- **`Inventory.py`**
- **`Player.py`**
- **`Ultilities.py`**

## Contribution
- **Huynh Gia Phong Tat**: Core mechanics and storyline.
- **Namho Kye**: Item system, combat logic, json file editing, menu configurations.